In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
from Data import Data 
import matplotlib.pyplot as plt
import nvector as nv
import osmapi as osm
import networkx as nx

In [2]:
def calc_type_numeric(row):
    if row['type'] == "Car":
        val = 0
    if row['type'] == "Taxi":
        val = 1

    return val

In [3]:
def nearest_graph_data(df, graph):
    df = __apply_parallel(df, __construct_graph_data_cols(graph))
    return df

In [4]:
def __construct_graph_data_cols(graph):
    def aux(row):
        coord = (row['lat'],row['lon'])
        nn = ox.get_nearest_node(graph, coord)
        start, end, _ = ox.get_nearest_edge(graph, coord)
        if start > end:
            start, end = end, start
        edge_prog = __edge_progress(graph, start, end, coord)
        row['nearest_node'] = nn
        row['nearest_edge_start_node'] = start
        row['nearest_edge_end_node'] = end
        row['edge_progress']  = edge_prog
        return row
    return aux

In [5]:
def prepare_df(df):
    df = df.drop(df[(df.type != "Car") & (df.type != "Taxi")].index)
    df['type_numeric'] = df.apply(calc_type_numeric, axis=1)
    df2 = df.reset_index().groupby(['id'])['time'].count()
    df2 = df2.drop(df2[df2 > 30].index)
    df = df.drop(df2.index)
    return df

In [6]:
df = Data('sample.csv').df
print(prepare_df(df))

               lat        lon    speed  lon_acc  lat_acc type  traveled_d  \
id time                                                                     
2  0.00  37.979775  23.733423  24.8006   0.0000   0.0000  Car        7.99   
   0.04  37.979778  23.733422  24.8006  -0.0000  -0.0000  Car        7.99   
   0.08  37.979780  23.733420  24.8006   0.0000   0.0000  Car        7.99   
   0.12  37.979782  23.733418  24.8006  -0.0000  -0.0000  Car        7.99   
   0.16  37.979784  23.733417  24.8006   0.0000   0.0000  Car        7.99   
...            ...        ...      ...      ...      ...  ...         ...   
9  2.84  37.979802  23.733362  37.4951   0.0222  -0.0207  Car       29.88   
   2.88  37.979805  23.733359  37.4989   0.0148  -0.0195  Car       29.88   
   2.92  37.979808  23.733357  37.5021   0.0101  -0.0183  Car       29.88   
   2.96  37.979811  23.733354  37.5046   0.0022  -0.0171  Car       29.88   
   3.00  37.979814  23.733351  37.5060  -0.0096  -0.0159  Car       29.88   